## Advent of code 2019 day 11-20
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import heapq
import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line=line.strip()
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2019 day 19
# start_ts=1668239345
# mv ~/Downloads/input* data_src/2019-day-19-input.txt
# big input file looks like: IntCode
# idea: part 1 just run intcode in the given range

class Computer:
    def __init__(self):
        self.relbase=0
        self.i_ptr=0
        self.OFFSET_DIVS={1: 100, 2: 1000, 3: 10000}
        self.board=None

    def fetch_param(self, data, opcode, i, offset):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            return data[param]
        elif mode==1: # immediate mode
            return param
        elif mode==2: # relative mode
            return data[param+self.relbase]
        else:
            print(f'invalid mode {mode}')
            assert False

    def store_param(self, data, opcode, i, offset, newval):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            data[param]=newval
        elif mode==2: # relative mode
            data[param+self.relbase]=newval
        else:
            print(f'invalid mode {mode}')
            assert False

    def run_opcodes(self, data): # HALTS ON INPUT -1 !
        i=self.i_ptr
        halted=False
        while True:
            if data[i]%100==1: # add
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a+b)
                i+=4
            elif data[i]%100==2: # mult
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a*b)
                i+=4
            elif data[i]%100==3: # input
                a=self.get_input()
                if a==-1:
                    halted=True
                    break
                self.store_param(data, data[i], i, 1, a)
                i+=2
            elif data[i]%100==4: # output
                a=self.fetch_param(data, data[i], i, 1)
                self.put_output(a)
                i+=2
            elif data[i]%100==5: # jump-if-true
                a=self.fetch_param(data, data[i], i, 1)
                if a!=0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==6: # jump-if-false
                a=self.fetch_param(data, data[i], i, 1)
                if a==0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==7: # less-than
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a<b else 0)
                i+=4
            elif data[i]%100==8: # equals
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a==b else 0)
                i+=4
            elif data[i]%100==9: # relbase
                a=self.fetch_param(data, data[i], i, 1)
                self.relbase+=a
                i+=2
            elif data[i]%100==99: # stop
                halted=True
                break
            else:
                print(f'unknown instruction {data[i]} at position {i}')
                assert False
        self.i_ptr=i
        return

    def init_board(self, do_part):
        assert self.board is None
        self.board={} # maps (x,y) cell to 0/1
        self.do_part=do_part
        self.part1_input=[]
        self.part1_input_i=0
        self.part1res=None

    def get_input(self):
        if self.board is None:
            self.init_board()
        if self.do_part==1:
            res=self.part1_input[self.part1_input_i]
            self.part1_input_i+=1
            return res
        else:
            assert self.do_part==2
            return None

    def put_output(self, a):
        if self.board is None:
            self.init_board()
        if self.do_part==1:
            #self.board[ (self.px, self.py) ]=a
            #self.px+=1
            self.part1res=a
        else:
            pass

    def print_painted(self, noprint=False):
        y_vals=[tup[1] for tup in self.board.keys()]
        x_vals=[tup[0] for tup in self.board.keys()]
        res=[]
        for y in range(min(y_vals), max(y_vals)+1):
            row=''
            for x in range(min(x_vals), max(x_vals)+1):
                c=self.board.get( (x, y) , -1)
                row+=('#' if c==1 else '.')
            if not noprint:
                print(row)
            res.append(row)
        return res

def count_intersections(strs):
    res=0
    for y, row in enumerate(strs):
        for x in range(len(row)):
            if x>=1 and y>=1 and x<len(row)-1 and y<len(strs)-1 and \
             row[x-1:x+2]=='###' and strs[y-1][x]=='#' and strs[y+1][x]=='#':
                #print(f'inter {x},{y}')
                res+=x*y
    return res

sample1=open('data_src/2019-day-19-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ] # program

def paint_beam(w, h, noprint=False):
    '''use intocode to paint in given range, count and return'''
    do_part=1
    if not noprint:
        print(f'{do_part=}')
    part1_count=0
    board2={} # maps (px,py) to 0 or 1
    for px in range(w):
        for py in range(h):
            data=collections.defaultdict(lambda : 0)
            for i,v in enumerate(data0): # converted to large mem
                data[i]=v
            cmp=Computer()
            cmp.init_board(do_part)
            cmp.part1_input=[px, py]
            cmp.run_opcodes(data)
            assert cmp.part1res in {0, 1}
            part1_count+=cmp.part1res
            board2[(px,py)]=cmp.part1res
    if not noprint:
        print(f'{part1_count=}')
    return board2

board2=paint_beam(50, 50)

# part 1: 171

In [ ]:
# part 2
# idea: instead of painting the whole beam build it up on demand,
# then start at an ample size and just move the 100x100 sqare to minimum position
# (this didn't give the right answer for unclear reason, so instead used binary search to find the minimum
# painting size)

#cmp=Computer()
#cmp.board=board2
#cmp.print_painted()

def get_board_pixel(board, x, y):
    pos=(x,y)
    if pos in board:
        return board[pos]
    data=collections.defaultdict(lambda : 0)
    for i,v in enumerate(data0): # converted to large mem
        data[i]=v
    cmp=Computer()
    cmp.init_board(do_part=1)
    cmp.part1_input=[x, y]
    cmp.run_opcodes(data)
    assert cmp.part1res in {0, 1}
    board[pos]=cmp.part1res
    return board[pos]

def square_fits(board, x, y, sz):
    '''given upper left corner, does this size square fit?'''
    return get_board_pixel(board, x, y)==1 and get_board_pixel(board, x+sz-1, y)==1 and \
        get_board_pixel(board, x, y+sz-1)==1 and get_board_pixel(board, x+sz-1, y+sz-1)==1

def square_size(board, boardsz, size_override=None):
    '''given 0/1 board map up to boardsz big return largest square that will fit and its x,y as (size, x, y)'''
    # start on lowest row, leftmost pixel, try increasingly large squares, 
    # given square max size try to move it up or left as far as possible
    y0=boardsz-1
    x0=None
    for x in range(boardsz): # could be sped up with binary search
        if get_board_pixel(board, x, y0)==1:
            x0=x
            break
    # x0,y0 is first pixel on last row
    if x0 is None:
        return None
    square_max=1
    while True:
        sz=square_max+1
        lx=x0
        ty=y0-(sz-1)
        if square_fits(board, lx, ty, sz):
            square_max+=1
        else:
            break
    sz=square_max
    if size_override is not None:
        sz=size_override        
    lx=x0
    ty=y0-(sz-1)
    if not square_fits(board, lx, ty, sz):
        return None
    # sz, lx, ty are size and first pixel of lowest max square
    #print(sz, lx, ty)
    while True:
        if square_fits(board, lx-1, ty, sz):
            lx-=1
        elif square_fits(board, lx, ty-1, sz):
            ty-=1
        elif square_fits(board, lx-1, ty-1, sz):
            lx-=1
            ty-=1
        else:
            break
    return sz, lx, ty

# binary search for best painting/board size to result in a 100x100 square
start=1
end=5000
board3={} # persistent board
while start<end-1:
    mid=(start+end)//2
    square_tup=square_size(board3, mid)
    if square_tup is None or square_tup[0]<100:
        start=mid
    else:
        end=mid
for width in [start, end]:
    square_tup=square_size(board3, width)
    sz, lx, ty=square_tup
    print(width, sz, lx, ty, lx*10000+ty)

# part 2: 9741242

In [ ]:
# 2019 day 18
# start_ts=1667728161
#  (actual start at 1667719820, paused after part 1 at 1667734619, continued at 1667742960)
# mv ~/Downloads/input* data_src/2019-day-18-input.txt
# big input file looks like: a map
# idea: part 1 parse as board, then BFS to find shortest paths, BFS to iterate over keysets
# part 2: updated parent BFS with bigger state and todos (single keyset, multiple positions),
#  total dist remains a single number

sample1='''
#########
#b.A.@.a#
#########
''' # part 1, 8 steps

sample2='''
########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################
''' # part 1, 86 steps

sample3='''
########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################
''' # part 1, 132 steps

sample4='''
#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################
''' # part 1, 136

sample5='''
########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################
''' # part 1, 81 steps

def list_keys_startpos(board):
    '''given a board returns a map of key to pos and startpos as x0, y0'''
    keymap={}
    x0=None
    y0=None
    for y,row in enumerate(board):
        for x,c in enumerate(row):
            if c=='@':
                assert x0 is None
                x0=x
                y0=y
            elif 'a'<=c<='z':
                assert c not in keymap
                keymap[c]=(x,y)
    assert x0 is not None
    return keymap, x0, y0

def is_reachable_key(c):
    '''is board character c reachable? returns False, True or needed key'''
    if c=='.' or c=='@' or ('a'<=c<='z'):
        return True
    if 'A' <= c <= 'Z':
        return chr(ord(c)-ord('A')+ord('a'))
    return False

def find_dists_keys(board, x0, y0, keymap):
    '''given board and startpos x0,y0 try to reach all keys in keymap, with different keysets,
    return sorted list of target key, minimum distance, needed keys, uses BFS,
    (so also lists longer options with fewer keys needed)'''
    #assert is_reachable(board[y0][x0], keys)
    todos=set()
    reached={} # maps pos to list of (min dist., set of needed keys)
    todos.add( (x0, y0) )
    reached[(x0, y0)]=[(0, set())]
    while len(todos)>0:
        oldpos=todos.pop()        
        tx, ty=oldpos
        for x, y in [(tx+1, ty), (tx-1, ty), (tx, ty-1), (tx, ty+1)]:
            if x<0 or y<0 or x>=len(board[0]) or y>=len(board):
                continue
            key=is_reachable_key(board[y][x])
            if key==False:
                continue
            newpos=(x,y)
            # step from reached[oldpos] to reached[newpos]
            # looking at reached[oldpos] can go to newpos by adding key and 1 distance for each,
            # then have to compare to reached[newpos]; if set is the same just compare distance,
            # if set is bigger and distance is bigger skip this new option,
            # else add new option
            if newpos not in reached:
                reached[newpos]=[]
            changed=False
            for olddist, oldkeys in reached[oldpos]:
                dist=olddist+1
                if key==True:
                    keys=oldkeys
                else:
                    keys=set(oldkeys)
                    keys.add(key)
                merged=False
                for i, row in enumerate(reached[newpos]):
                    newdist, newkeys=row
                    if newkeys==keys:
                        if newdist>dist:
                            reached[newpos][i]=(dist, newkeys)
                            changed=True
                        merged=True
                        break
                    if len(keys-newkeys)>0 and dist>newdist: # worthless
                        merged=True
                        break
                if not merged:
                    reached[newpos].append( (dist, keys) )
                    changed=True
            if changed:
                todos.add(newpos)
    keyreachable=[]
    for key, keypos in keymap.items():
        if keypos in reached:
            for reachedrow in reached[keypos]:
                reacheddist, reachedkeys=reachedrow
            keyreachable.append( (key, reacheddist, reachedkeys) )
    keyreachable.sort(lambda tup: tup[1])
    return keyreachable

def keyset_to_str(keyset):
    keys=list(keyset)
    return ''.join(sorted(keys))

def find_min_total_bfs(board, x0, y0, keymap):
    '''given board, current pos x,y and given keymap give minimum distance
    to reach all keys, uses BFS on map of keyset,pos'''
    bestdist=None
    reached_with_keys={} # maps pos to find_dists_keys result
    reacheddist={} # maps (keystr, posx, posy) to best dist so far
    todos=set()
    rkey=(keyset_to_str(keymap.keys()), x0, y0)
    reacheddist[rkey]=0
    todos.add( rkey )
    while len(todos)>0:
        todo=todos.pop()
        todokeystr, x, y=todo
        assert len(todokeystr)>=1
        curpos=(x, y)
        if curpos not in reached_with_keys:
            reached_with_keys[curpos]=find_dists_keys(board, x, y, keymap)
        keyreachable=reached_with_keys[curpos]
        todo_keys_done=set()
        for keyreachrow in keyreachable:
            key, keydist, keysneeded=keyreachrow
            if (key not in todokeystr) or (key in todo_keys_done):
                continue
            todo_keys_done.add(key)
            neededfailed=False
            for kn in keysneeded:
                if kn in todokeystr:
                    neededfailed=True
                    break
            if neededfailed: # this option needs more keys than we have
                continue            
            keypos=keymap[key]
            assert keydist>=0
            newkeystr=todokeystr.replace(key, '')
            newdist=keydist+reacheddist[todo]
            if len(newkeystr)<1: # done
                if bestdist is None or newdist<bestdist:
                    bestdist=newdist
                continue
            newrkey=(newkeystr, keypos[0], keypos[1])
            if newrkey not in reacheddist or reacheddist[newrkey]>newdist:
                reacheddist[newrkey]=newdist
                todos.add(newrkey)
    print(f'created {len(reacheddist)=}, {len(reached_with_keys)=}')
    return bestdist

sample1=open('data_src/2019-day-18-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
board=lines
keymap, x0, y0=list_keys_startpos(board)
dist=find_min_total_bfs(board, x0, y0, keymap)
print(f'part 1 {dist=}')

# part 1: 6162

In [ ]:
# part 2

sample2='''
#######
#a.#Cd#
##...##
##.@.##
##...##
#cB#Ab#
#######
''' # part 2, 8 steps

sample3='''
###############
#d.ABC.#.....a#
######...######
######.@.######
######...######
#b.....#.....c#
###############
''' # part 2, 24 steps

sample4='''
#############
#g#f.D#..h#l#
#F###e#E###.#
#dCba...BcIJ#
#####.@.#####
#nK.L...G...#
#M###N#H###.#
#o#m..#i#jk.#
#############
''' # part 2, 72 steps

def list_keys_startpos_part2(board):
    '''given a board converts to part 2, returns a map of key to pos and list of startpos as x0, y0'''
    keymap={}
    startpos=[]
    for y,row in enumerate(board):
        pos=row.find('.@.')
        if pos>=0:
            board[y-1]=board[y-1][:pos]+'@#@'+board[y-1][pos+3:]
            board[y]=board[y][:pos]+'###'+board[y][pos+3:]
            board[y+1]=board[y+1][:pos]+'@#@'+board[y+1][pos+3:]
            break
    for y,row in enumerate(board):
        for x,c in enumerate(row):
            if c=='@':
                startpos.append( (x, y) )
            elif 'a'<=c<='z':
                assert c not in keymap
                keymap[c]=(x,y)
    assert len(startpos)==4
    return keymap, startpos

def find_min_total_bfs_part2(board, startpos, keymap):
    '''given board, startpos and keymap give minimum distance
    to reach all keys with parallel robots, uses BFS on map of keyset, pos1, pos2, pos3, pos4'''
    bestdist=None
    reached_with_keys={} # maps pos to find_dists_keys result
    reacheddist={} # maps (keystr, pos1x, pos1y, pos2x, pos2y, pos3x, pos3y, pos4x, pos4y) to best dist so far
    todos=set()
    rkey=[keyset_to_str(keymap.keys())]
    for pos in startpos:
        rkey.append(pos[0])
        rkey.append(pos[1])
    rkey=tuple(rkey)
    reacheddist[rkey]=0
    todos.add( rkey )
    while len(todos)>0:
        todo=todos.pop()
        todokeystr=todo[0]
        assert len(todokeystr)>=1
        for todoi in range(1, len(todo), 2): # one by one step each bot
            curpos=(todo[todoi], todo[todoi+1])
            if curpos not in reached_with_keys:
                reached_with_keys[curpos]=find_dists_keys(board, curpos[0], curpos[1], keymap)
            keyreachable=reached_with_keys[curpos]
            todo_keys_done=set()
            for keyreachrow in keyreachable:
                key, keydist, keysneeded=keyreachrow
                if (key not in todokeystr) or (key in todo_keys_done):
                    continue
                todo_keys_done.add(key)
                neededfailed=False
                for kn in keysneeded:
                    if kn in todokeystr:
                        neededfailed=True
                        break
                if neededfailed: # this option needs more keys than we have
                    continue            
                keypos=keymap[key]
                assert keydist>=0
                newkeystr=todokeystr.replace(key, '')
                newdist=keydist+reacheddist[todo]
                if len(newkeystr)<1: # done
                    if bestdist is None or newdist<bestdist:
                        bestdist=newdist
                    continue
                newrkey=[newkeystr]
                for newi in range(1, len(todo), 2):
                    if newi==todoi:
                        newrkey.append(keypos[0])
                        newrkey.append(keypos[1])
                    else:
                        newrkey.append(todo[newi])
                        newrkey.append(todo[newi+1])
                newrkey=tuple(newrkey)
                if newrkey not in reacheddist or reacheddist[newrkey]>newdist:
                    reacheddist[newrkey]=newdist
                    todos.add(newrkey)
    print(f'created {len(reacheddist)=}, {len(reached_with_keys)=}')
    return bestdist

sample1=open('data_src/2019-day-18-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
board=lines
keymap, startpos=list_keys_startpos_part2(board)
dist=find_min_total_bfs_part2(board, startpos, keymap)
print(f'part 2 {dist=}')

# part 2: 1556

In [ ]:
# part 1 old

def is_reachable(c, keys):
    '''is board character c reachable with given set of keys?'''
    if c=='.' or c=='@' or ('a'<=c<='z'):
        return True
    if ('A' <= c <= 'Z') and chr(ord(c)-ord('A')+ord('a')) in keys:
        return True
    return False
    
def find_dists(board, x0, y0, keys, todo_keys, keymap):
    '''given board and startpos x0,y0 try to reach all todo_keys with the given set of keys,
    return sorted list of todo key, minimum distance (only for reachable keys), uses BFS'''
    assert is_reachable(board[y0][x0], keys)
    todos=set()
    reached={} # maps pos to min dist.
    todos.add( (x0, y0) )
    reached[(x0, y0)]=0
    while len(todos)>0:
        tx, ty=todos.pop()
        tdist=reached[(tx,ty)]
        for x, y in [(tx+1, ty), (tx-1, ty), (tx, ty-1), (tx, ty+1)]:
            if x<0 or y<0 or x>=len(board[0]) or y>=len(board):
                continue
            if not is_reachable(board[y][x], keys):
                continue
            newpos=(x,y)
            if newpos not in reached or reached[newpos]>tdist+1:
                reached[newpos]=tdist+1
                todos.add(newpos)
    keyreachable=[]
    for key in todo_keys:
        keypos=keymap[key]
        if keypos in reached:
            keyreachable.append( (key, reached[keypos]) )
    keyreachable.sort(lambda tup: tup[1])
    return keyreachable

#bestdistsofar=None # used for DFS pruning
#reached_with_keys={} # maps pos to find_dists_keys output
#dist=find_min_total(board, x0, y0, set(), set(keymap.keys()), keymap, 0)
def find_min_total(board, x, y, keys, todo_keys, keymap, dist):
    '''given board, current pos x,y and given set of keys give minimum distance
    to reach remaining todo_keys, already traveled dist, uses DFS'''
    #print(f'find_min_total {x=}, {y=}, {keys=}, {todo_keys=}, {dist=}')
    global bestdistsofar, reached_with_keys
    if len(todo_keys)<1:
        if bestdistsofar is None or bestdistsofar>dist:
            bestdistsofar=dist
            print(f'find_min_total {bestdistsofar=}')
        return dist
    bestdist=None
    curpos=(x, y)
    if curpos not in reached_with_keys:
        reached_with_keys[curpos]=find_dists_keys(board, x, y, keymap)
    keyreachable=reached_with_keys[curpos]
    todo_keys_done=set()
    for keyreachrow in keyreachable:
        key, keydist, keysneeded=keyreachrow
        if len(keysneeded-keys)>0: # this option needs more keys than we have
            continue
        if (key not in todo_keys) or (key in todo_keys_done):
            continue
        todo_keys_done.add(key)
        keypos=keymap[key]
        assert keydist>=0
        if bestdistsofar is not None and dist+keydist>=bestdistsofar: # prune
            continue
        keys.add(key)
        todo_keys.remove(key)
        nextdist=find_min_total(board, keypos[0], keypos[1], keys, todo_keys, keymap, dist+keydist)
        if nextdist is not None and (bestdist is None or nextdist<bestdist):
            bestdist=nextdist
        keys.remove(key)
        todo_keys.add(key)
    return bestdist

In [ ]:
# 2019 day 17
# start_ts=1667095749
#  actually started at 1666333849, paused at 1666339739, continued at 1667028656, paused at 1667038694,
#  continued at 1667046286, paused at 1667053136, continued at 1667118527, completed 1667129432
# mv ~/Downloads/input* data_src/2019-day-17-input.txt
# big input file looks like: intcode
# idea: part 1 run program, capture ascii, calculate intersections where 4 #s surround

sample1_strs='''
..#..........
..#..........
#######...###
#.#...#...#.#
#############
..#...#...#..
..#####...^..
''' # part 1

sample2_strs='''
''' # part 1 real input

sample3_strs='''
#######...#####
#.....#...#...#
#.....#...#...#
......#...#...#
......#...###.#
......#.....#.#
^########...#.#
......#.#...#.#
......#########
........#...#..
....#########..
....#...#......
....#...#......
....#...#......
....#####......
''' # part 2

class Computer:
    def __init__(self):
        self.relbase=0
        self.i_ptr=0
        self.OFFSET_DIVS={1: 100, 2: 1000, 3: 10000}
        self.board=None

    def fetch_param(self, data, opcode, i, offset):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            return data[param]
        elif mode==1: # immediate mode
            return param
        elif mode==2: # relative mode
            return data[param+self.relbase]
        else:
            print(f'invalid mode {mode}')
            assert False

    def store_param(self, data, opcode, i, offset, newval):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            data[param]=newval
        elif mode==2: # relative mode
            data[param+self.relbase]=newval
        else:
            print(f'invalid mode {mode}')
            assert False

    def run_opcodes(self, data): # HALTS ON INPUT -1 !
        i=self.i_ptr
        halted=False
        while True:
            if data[i]%100==1: # add
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a+b)
                i+=4
            elif data[i]%100==2: # mult
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a*b)
                i+=4
            elif data[i]%100==3: # input
                a=self.get_input()
                if a==-1:
                    halted=True
                    break
                self.store_param(data, data[i], i, 1, a)
                i+=2
            elif data[i]%100==4: # output
                a=self.fetch_param(data, data[i], i, 1)
                self.put_output(a)
                i+=2
            elif data[i]%100==5: # jump-if-true
                a=self.fetch_param(data, data[i], i, 1)
                if a!=0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==6: # jump-if-false
                a=self.fetch_param(data, data[i], i, 1)
                if a==0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==7: # less-than
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a<b else 0)
                i+=4
            elif data[i]%100==8: # equals
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a==b else 0)
                i+=4
            elif data[i]%100==9: # relbase
                a=self.fetch_param(data, data[i], i, 1)
                self.relbase+=a
                i+=2
            elif data[i]%100==99: # stop
                halted=True
                break
            else:
                print(f'unknown instruction {data[i]} at position {i}')
                assert False
        self.i_ptr=i
        return

    def init_board(self, do_part):
        assert self.board is None
        self.board={} # maps (x,y) cell to ascii code
        self.px=0
        self.py=0
        self.do_part=do_part
        self.part2plan=None
        self.part2planpos=0
        self.part2res=None

    def get_input(self):
        if self.board is None:
            self.init_board()
        if self.do_part==1:
            return -1
        else:
            assert self.do_part==2
            c=self.part2plan[self.part2planpos]
            self.part2planpos+=1
            return ord(c)

    def put_output(self, a):
        if self.board is None:
            self.init_board()
        if a==10:
            self.px=0
            self.py+=1
        else:
            self.board[ (self.px, self.py) ]=a
            self.px+=1
        self.part2res=a

    def print_painted(self, noprint=False):
        y_vals=[tup[1] for tup in self.board.keys()]
        x_vals=[tup[0] for tup in self.board.keys()]
        res=[]
        for y in range(min(y_vals), max(y_vals)+1):
            row=''
            for x in range(min(x_vals), max(x_vals)+1):
                c=self.board.get( (x, y) , -1)
                row+=chr(c) if c>=32 else '?'
            if not noprint:
                print(row)
            res.append(row)
        return res

def count_intersections(strs):
    res=0
    for y, row in enumerate(strs):
        for x in range(len(row)):
            if x>=1 and y>=1 and x<len(row)-1 and y<len(strs)-1 and \
             row[x-1:x+2]=='###' and strs[y-1][x]=='#' and strs[y+1][x]=='#':
                #print(f'inter {x},{y}')
                res+=x*y
    return res

#sample1_strs_b=[s for s in sample1_strs.splitlines() if len(s)>0 ]
#alsum=count_intersections(sample1_strs_b)
#print(f'{alsum=}')

sample3_strs_b=[s for s in sample3_strs.splitlines() if len(s)>0 ]

part2sol='A,B,A,B,C,C,B,A,B,C/L,10,R,10,L,10,L,10/R,10,R,12,L,12/R,12,L,12,R,6'

sample1=open('data_src/2019-day-17-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ] # program
for do_part in [1,2]:
    print(f'{do_part=}')
    data=collections.defaultdict(lambda : 0)
    for i,v in enumerate(data0): # converted to large mem
        data[i]=v
    cmp=Computer()
    cmp.init_board(do_part)
    if do_part==2:
        data[0]=2
        cmp.part2plan=part2sol.replace('/', '\n')+'\n'+'n\n'
        print(f'part2plan: {cmp.part2plan}')
    cmp.run_opcodes(data)
    if do_part==1:
        sample2_strs=cmp.print_painted(noprint=True)
        #print(collections.Counter(cmp.board.values()))
        alsum=count_intersections(sample2_strs)
        print(f'part 1 {alsum=}')
    else:
        assert do_part==2
        print(f'collected dust: {cmp.part2res}')

# part 1: 7720
# part 2: 1681189


In [ ]:
# part 2 A
# general idea is A* search, generating and testing plans that leave less and less of the map unreached,
# initially tried to generate full 4-part plans immediately (including subroutines), but this didn't work
# so well, hard to efficiently explore the whole search space that way, so instead now
# first generating solutions in a single routine, then splitting up below (which turned out to be easier
# than it initially looked)
# key part of the solution is next_intersect, which generates all steps in a certain direction to next
# intersections (or the end), if you just take the closest step you won't find short solutions and searching will
# take much longer.
# plan_move_opt and plan_main are not really needed in this solution

def test_pos(board, x, y):
    '''returns boolean, is pos on scaffold?'''
    if x<0 or y<0 or y>=len(board) or x>=len(board[y]):
        return False
    c=board[y][x]
    return c=='#' or c=='+'

def try_plan_unreached(board, pos, direc, plan):
    '''gives plan as main routine/A/B/C, does robot drop off (return -1), or else how many
    cells remain unreached? pos is (x, y), direc is 0=up, 1=right, 2=down, 3=left,
    if robot reaches any position more than 4 times we assume a loop, counts as drop off (-2),
    returns unreached, endpos, end-direc'''
    # in board changes # to + when reached
    assert isinstance(board, list) and len(board)>1
    board=list(board)
    x,y=pos
    plan=plan.split('/')
    reached=collections.Counter() # (x,y) to count of reached
    for routine in plan[0].split(','):
        if not routine:
            continue
        findex={'A': 1, 'B': 2, 'C': 3}[routine]
        for cmd in plan[findex].split(','):
            if not cmd:
                continue
            if cmd=='R':
                direc=(direc+1)%4
            elif cmd=='L':
                direc=(direc-1)%4
            else:
                dist=int(cmd)                
                if not test_pos(board, x, y): # start pos
                    return -1, None, None
                if (x, y) not in reached:
                    reached[(x, y)]=1
                board[y]=board[y][:x]+'+'+board[y][x+1:]
                for _ in range(dist):
                    if direc==0:
                        y-=1
                    elif direc==1:
                        x+=1
                    elif direc==2:
                        y+=1
                    else:
                        assert direc==3
                        x-=1
                    if not test_pos(board, x, y):
                        return -1, None, None
                    reached[(x, y)]+=1
                    if reached[(x, y)]>4:
                        return -2, None, None
                    board[y]=board[y][:x]+'+'+board[y][x+1:]
    unreached=0
    for row in board:
        for c in row:
            if c=='#':
                unreached+=1
            else:
                assert c=='+' or c=='.'
    pos=(x, y)
    return unreached, pos, direc

def init_board(board):
    '''given board return pos and dir and board'''
    assert isinstance(board, list) and len(board)>1
    board=list(board)
    pos=None
    direc=None
    for y, row in enumerate(board):
        for x, c in enumerate(row):
            if c=='#' or c=='.':
                continue
            if c=='^':
                direc=0
            elif c=='>':
                direc=1
            elif c=='v':
                direc=2
            elif c=='<':
                direc=3
            else:
                assert False, f'unexpected char {c}'
            assert pos is None
            pos=(x, y)
    assert pos is not None
    row=board[pos[1]]
    x=pos[0]
    board[pos[1]]=row[:x]+'#'+row[x+1:]
    return pos, direc, board

def next_intersect(board, pos, direc):
    '''given pos and dir return set of (dist, newreached),
    dist is steps to next intersection, if None returned we fell off, 
    newreached is True if reaching a new cell, False otherwise'''
    x,y=pos
    dist=0
    newreached=False
    if not test_pos(board, x, y): # start pos
        return None
    res=set()
    while True:
        if direc==0:
            y-=1
        elif direc==1:
            x+=1
        elif direc==2:
            y+=1
        else:
            assert direc==3
            x-=1
        if not test_pos(board, x, y):
            break
        dist+=1
        if board[y][x]=='#':
            newreached=True
        # count scaffold around
        count=0
        if y>=1 and (board[y-1][x]=='#' or board[y-1][x]=='+'):
            count+=1
        row=board[y]
        if x<len(row)-1 and (row[x+1]=='#' or row[x+1]=='+'):
            count+=1
        if y<len(board)-1 and (board[y+1][x]=='#' or board[y+1][x]=='+'):
            count+=1
        if x>=1 and (row[x-1]=='#' or row[x-1]=='+'):
            count+=1
        if count==1 or count>2:
            res.add( (dist, newreached) )
    if dist>0:
        res.add( (dist, newreached) )
    return res
            
def plan_move_opt(plan, routine, direcdist, dist, override20=False):
    '''routine 1-3, extend with turning direcdist steps (turning right) and moving dist'''
    assert dist>0
    assert routine>=1
    plan=plan.split('/')
    # extend either to the last routine in main (or A), or to the next unused routine
    mainlastroutine=1
    if len(plan[0])>0:
        mainlastroutine=ord(plan[0][-1])-ord('A')+1
    if routine!=mainlastroutine and not (len(plan[routine])<1 and routine>1 and len(plan[routine-1])>0):
        return None
    row=plan[routine]
    lastcmd=None
    if len(row)>0:
        lastcmd=row.split(',')[-1]
    assert lastcmd!='R' and lastcmd!='L' # because after turning we always move
    if direcdist==1: 
        if len(row)>0:
            row+=','
        row+='R'
    elif direcdist==2: # for now 180 degree turn will always be L,L
        if len(row)>0:
            row+=','
        row+='L,L'
    elif direcdist==3: 
        if len(row)>0:
            row+=','
        row+='L'
    else:
        assert direcdist==0
    lastcmd=None
    if len(row)>0:
        lastcmd=row.split(',')[-1]
    if lastcmd and lastcmd!='R' and lastcmd!='L': # add our distance to lastcmd, going in same direction
        i=row.rfind(',') # -1 if not found
        lastdist=int(row[i+1:])
        assert int(lastcmd)==lastdist
        row=row[:i+1]+str(lastdist+dist)
    else:
        if len(row)>0:
            row+=','
        row+=str(dist)
    if len(row)>20 and not override20:
        return None
    plan[routine]=row
    return '/'.join(plan)

def plan_main(plan, routine):
    '''depending on routine extend main routine'''
    mprtntoi={1: 'A', 2: 'B', 3: 'C'}
    if not plan:
        return None
    plan=plan.split('/')
    if routine!=0 and len(plan[routine])<=0:
        return None
    # make sure all used routines are set at least once
    row=plan[0]
    for r2 in range(1, 4):
        c=mprtntoi[r2]
        if len(plan[r2])>0 and row.find(c)<0:
            if len(row)>0:
                row+=','
            row+=c
    if routine>0:
        if len(row)>0:
            row+=','
        row+=mprtntoi[routine]
    if len(row)>20:
        return None
    plan[0]=row
    return '/'.join(plan)

# A* search - heapq of todos consisting of (unreached+len(plan), plan, unreached, pos_x, pos_y, direc)
#  where plan is main / A / B / C and the rest of the tuple are the result of trying the plan
# take first todo, now for all 4 directions try next_intersect, if ok add to A, B or C
# also generate todos for longer mains
# when finally found print the plan
# update: this code does not correctly exhaustively search the full plan search space,
# for now search for unlimited single-routine plans

def search(board):
    pos0, direc0, board=init_board(board)
    print(f'init {pos0=}, {direc0=}')
    todos=[]
    plan1='A///'
    unreached1, pos1, direc1=try_plan_unreached(board, pos0, direc0, plan1)
    tried=set() # set of all tried (optimized) plans
    tried.add(plan1)
    found=set() # set of all succesful plans
    assert unreached1!=0 # otherwise would already be done
    print(f'plan1 {unreached1=}, {pos1=}, {direc1=}, {plan1=}')
    heapq.heappush(todos, (unreached1+len(plan1), plan1, unreached1, pos1[0], pos1[1], direc1) )
    pcount=0
    while len(todos)>0:
        todo=heapq.heappop(todos)
        score, plan1, unreached1, pos1_x, pos1_y, direc1=todo
        directions=[]
        for direcdist in range(4):
            direc1b=(direc1+direcdist)%4
            intersects=next_intersect(board, (pos1_x, pos1_y), direc1b)
            if not intersects:
                continue
            for dist, newreached in intersects:
                assert isinstance(newreached, bool)
                directions.append( (direcdist, dist, newreached) )
        directions.sort(key=lambda tup: tup[1]-(1000 if tup[2] else 0))
        for direcdist, dist, newreached in directions:
            routine=1
            plan2=plan_move_opt(plan1, routine, direcdist, dist, override20=True)
            if plan2 is None:
                continue
            unreached2, pos2, direc2=try_plan_unreached(board, pos0 , direc0, plan2)
            tried.add(plan2)
            if unreached2<0:
                continue
            if unreached2==0:
                found.add(plan2)
                splitplan=search_plan_main(plan2) # code below
                if splitplan:
                    print(f'FOUND! {splitplan}')
                if len(found)>50000:
                    return found
            pcount+=1
            if pcount<20 or pcount%100000==0:
                print(f'plan2 {pcount=}, found={len(found)}, {unreached2=}, {pos2=}, {direc2=}, {plan2=}')
            heapq.heappush(todos, (unreached2+len(plan2), plan2, unreached2, pos2[0], pos2[1], direc2) )
    print(f'FAILED, tried {len(tried)} plans')

In [ ]:
# part 2 B

#for row in sample3_strs_b:
#    print(row)
#found=search(sample3_strs_b)
#for row in sample2_strs:
#    print(row)
found=search(sample2_strs)
#with open('data_src/2019-day-17-found.txt', 'w') as f:
#    json.dump(sorted(found), f)

In [ ]:
# part 2 C

# divide into three repeating subsequences - each item in str can be marked as belonging to either A, B or C,
# which also determines the overall pattern, DFS, fast enough by maintaining the 20 char limit,
# so can prune pretty strongly, NB first char is always A, next can also be B, if that is used can also be C
# result: indeed fast enough

class Routine():
    def __init__(self):
        self.items=[]
        self.chars=0 # length in chars
        self.closed=False

    def fit(self, item, add=False):
        '''check that item fits and optionally add'''
        assert not self.closed
        s=str(item)
        if self.chars>0:
            s=','+s
        if self.chars+len(s)>20:
            return False
        if add:
            self.items.append(item)
            self.chars+=len(s)
        return True

    def pop(self):
        '''remove last item'''
        assert (not self.closed) and len(self.items)>0
        item=self.items.pop()
        s=str(item)
        self.chars-=len(s)
        if len(self.items)>0:
            self.chars-=1 # there was a separator comma

def plan_str(plan, ci=False):
    s=''
    for r in plan:
        if len(s)>0:
            s+='/'
        if ci and r.closed:
            s+='['
        s+= ','.join(r.items)
        if ci and r.closed:
            s+=']'
    return s

def search_plan(seq, seqpos, plan, routine, rpos):
    '''split seq up into 3 repeating subroutines, currently at item seqpos, plan is a list of Routine 
    (first for main, then 3 subroutines), routine is index 1-3, 
    rpos is index in Routine.items of last matched item, DFS'''
    #print(f'{seqpos=}, {plan_str(plan, ci=True)}, {routine=}, {rpos=}')
    if seqpos>=len(seq):
        return True
    # continue with current routine, or (if open first close current routine) switch to start of any
    # closed routine, or start new open routine after last closed one
    item=seq[seqpos]
    r=plan[routine]
    if not r.closed and r.fit(item): # continue with current open routine
        assert r.fit(item, add=True)
        found=search_plan(seq, seqpos+1, plan, routine, rpos)
        if found:
            return True
        r.pop()
    if r.closed and rpos+1<len(r.items) and r.items[rpos+1]==item: # continue with current closed routine
        rpos+=1
        found=search_plan(seq, seqpos+1, plan, routine, rpos)
        if found:
            return True
        rpos-=1
    # close current routine
    if not r.closed and len(r.items)<1:
        return False # can't close empty routine
    savedr=r
    savedrclosed=r.closed
    r.closed=True
    # switch to start of a closed routine
    for newroutine in range(1, 4):
        if plan[newroutine].closed and item==plan[newroutine].items[0] and plan[0].fit('X'): # can switch
            assert plan[0].fit(chr(ord('A')+newroutine-1), add=True)
            found=search_plan(seq, seqpos+1, plan, newroutine, 0)
            if found:
                return True
            plan[0].pop()
    # start new open routine after the last one
    for newroutine in range(1, 4):
        if plan[newroutine].closed==False and plan[0].fit('X') and plan[newroutine].fit(item):
            assert len(plan[newroutine].items)<1
            assert plan[0].fit(chr(ord('A')+newroutine-1), add=True)
            assert plan[newroutine].fit(item, add=True)
            found=search_plan(seq, seqpos+1, plan, newroutine, -1)
            if found:
                return True
            plan[0].pop()
            plan[newroutine].pop()
            break
    # unclose current routine
    savedr.closed=savedrclosed
    return False

def search_plan_main(seqs0):
    assert seqs0.startswith('A/')
    assert seqs0.endswith('//')
    seqs=seqs0[2:-2]
    seq=seqs.split(',')
    #print(f'{len(seq)=}, {seqs}')
    plan=[]
    while len(plan)<4:
        plan.append(Routine())
    assert plan[0].fit('A', add=True)
    ok=search_plan(seq, 0, plan, 1, -1)
    if ok:
        return plan_str(plan)
    else:
        return None

In [ ]:
# part 2 D, actually not needed after improving searching

found=set()
for filename in ['data_src/2019-day-17-found.txt', 'data_src/2019-day-17-found2.txt']:
    with open(filename) as f:
        items=json.load(f)
        for item in items:
            found.add(item)
found=sorted(found, key=lambda x: len(x))
print('first:', found[0])
print('len:', len(found))
for seqs0 in found:
    res=search_plan_main(seqs0)
    if res:
        print(res)


In [ ]:
# 2019 day 16
# start_ts=1665896000
# (NB looked at other answers to find part 2 solution, so ranking irrelevant)
# mv ~/Downloads/input* data_src/2019-day-16-input.txt
# big input file looks like: single line of 650 digits
# idea: part 1 parse, then follow recipe
# part 2: see below

sample1='''
12345678
'''

sample2='''
80871224585914546619083218645595
'''

sample3='''
69317163492948606335995924319873
'''

sample4='''
03036732577212944063491565474664
'''

sample1=open('data_src/2019-day-16-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ [ int(x) for x in list(s) ] for s in lines ]
data=data[0]

def genpat(n):
    # return each elem in pat n+1 times, but skip first 1
    pat=[0, 1, 0, -1]
    cnt=0
    skip=1
    while True:
        for p in pat:
            for _ in range(n+1):
                if cnt<skip:
                    cnt+=1
                    continue
                yield p
                cnt+=1

def fft(inp):
    res=[]
    for i in range(len(inp)):
        p=genpat(i)
        elem=0
        for j in range(len(inp)):
            a=inp[j]
            b=next(p)
            if b==0:
                pass
            elif b== -1:
                elem-=a
            elif b==1:
                elem+=a
            else:
                assert False
        res.append(abs(elem)%10)
    return res

def lcm(a, b):
    return a*b//math.gcd(a, b)

# part 1
data1=list(data)
print(f'{len(data1)=}')
for x in range(100):
    data1=fft(data1)
offset=0
print('msg:', ''.join([ str(i) for i in data1[offset:offset+8] ]))

In [ ]:
# part 2 explore patterns, suppose input 8 long, repeated twice
# (actually cheated and looked at https://www.reddit.com/r/adventofcode/comments/ebai4g/2019_day_16_solutions/ 
# to figure this out, although it's pretty simple after the fact)

for i in range(16):
    p=genpat(i)
    gp=[]
    while len(gp)<16:
        gp.append(next(p))
    print(i, ':', gp)

# so, you can generate the second half of each input fft phase using just partial sums,
# iterating backward from the end, and if the offset is in the second half that's all you need

In [ ]:
# part 2 implementation

offset=int(''.join([str(i) for i in data[:7] ]))
data1=list(data * 10000)
full_len=len(data1)
print(f'{offset=}, {full_len=}')
assert offset>=full_len//2 # otherwise no fast solution
data1=data1[full_len//2:] # only second half
offset-=full_len//2

def fft2(inp):
    res=[]
    while len(res)<len(inp):
        res.append(0)
    sm=0
    for i in range(len(inp)-1, -1, -1):
        sm+=inp[i]
        res[i]=sm%10
    return res

for x in range(100):
    data1=fft2(data1)

print('msg:', ''.join([ str(i) for i in data1[offset:offset+8] ]))

# part 2: 82994322 after 8 s.

In [ ]:
# 2019 day 15
# start_ts=1665851479
# mv ~/Downloads/input* data_src/2019-day-15-input.txt
# big input file looks like: IntCode
# idea: part 1 BFS with a movement routine, first 1 step from start, then 2 steps etc.
# part 2: first tried to restart intcode to continue from oxygen position and re-explore, way too much hassle,
# in the end hacked find_route into fill_oxygen which was quite simple

class Computer:
    def __init__(self):
        self.relbase=0
        self.i_ptr=0
        self.OFFSET_DIVS={1: 100, 2: 1000, 3: 10000}
        self.board=None

    def fetch_param(self, data, opcode, i, offset):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            return data[param]
        elif mode==1: # immediate mode
            return param
        elif mode==2: # relative mode
            return data[param+self.relbase]
        else:
            print(f'invalid mode {mode}')
            assert False

    def store_param(self, data, opcode, i, offset, newval):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            data[param]=newval
        elif mode==2: # relative mode
            data[param+self.relbase]=newval
        else:
            print(f'invalid mode {mode}')
            assert False

    def run_opcodes(self, data): # HALTS ON INPUT -1 !
        i=self.i_ptr
        halted=False
        while True:
            if data[i]%100==1: # add
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a+b)
                i+=4
            elif data[i]%100==2: # mult
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a*b)
                i+=4
            elif data[i]%100==3: # input
                a=self.get_input()
                if a==-1:
                    halted=True
                    break
                self.store_param(data, data[i], i, 1, a)
                i+=2
            elif data[i]%100==4: # output
                a=self.fetch_param(data, data[i], i, 1)
                self.put_output(a)
                i+=2
            elif data[i]%100==5: # jump-if-true
                a=self.fetch_param(data, data[i], i, 1)
                if a!=0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==6: # jump-if-false
                a=self.fetch_param(data, data[i], i, 1)
                if a==0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==7: # less-than
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a<b else 0)
                i+=4
            elif data[i]%100==8: # equals
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a==b else 0)
                i+=4
            elif data[i]%100==9: # relbase
                a=self.fetch_param(data, data[i], i, 1)
                self.relbase+=a
                i+=2
            elif data[i]%100==99: # stop
                halted=True
                break
            else:
                print(f'unknown instruction {data[i]} at position {i}')
                assert False
        self.i_ptr=i
        return

    def init_board(self, do_part, startpos=(0, 0)):
        assert self.board is None
        self.board={} # maps (x,y) cell to status code
        self.bdist={} # maps (x,y) to shortest distance to reach this cell from (0,0)
        self.todo=set() # (x,y, x2, y2) positions still to explore as source and dest of step
        self.route=[] # list of (x,y) of current movement/explore
        self.oxypos=None # (x,y) of oxy
        self.curpos=startpos # (x,y) of current pos
        self.bdist[self.curpos]=0
        self.nextpos=None # set while moving (from input to output)
        self.todo_dist=1 # exploring 1 distance, i.e. the first step
        self.do_part=do_part # 1 or 2
        self.todo.add( (0,0,0,1) )
        self.todo.add( (0,0,1,0) )
        self.todo.add( (0,0,0,-1) )
        self.todo.add( (0,0,-1,0) )

    def get_input(self):
        if self.board is None:
            self.init_board()
        assert self.nextpos is None
        # can halt with -1
        # phase 1: follow route, if route empty pop todo & set route, if todo is empty - if found halt else
        # next distance step
        if len(self.route)<1:
            if len(self.todo)<1:
                if self.do_part==1 and self.oxypos is not None: # found oxy so done
                    return -1
                # generate new step of todos (first distance 1, then distance 2 etc.)
                assert max(self.bdist.values())==self.todo_dist
                for pos, dist in self.bdist.items():
                    if dist!=self.todo_dist:
                        continue
                    assert pos in self.board
                    for newpos in [(pos[0], pos[1]+1), (pos[0], pos[1]-1), (pos[0]-1, pos[1]), (pos[0]+1, pos[1])]:
                        if newpos in self.board:
                            continue
                        self.todo.add( (pos[0], pos[1], newpos[0], newpos[1]) )
                self.todo_dist+=1
            if self.do_part==2 and len(self.todo)<1:
                return -1
            while len(self.todo)>0:
                nexttodo=self.todo.pop()
                self.route=self.find_route(self.curpos, tuple(nexttodo[:2]))
                if self.do_part==2 and self.route is None: # sometimes in part 2 route cannot be found?
                    continue
                self.route.append(tuple(nexttodo[2:]))
                break
            if self.route is None or len(self.route)<1:
                if do_part==1:
                    assert False
                else:
                    return -1
        # follow route
        self.nextpos=self.route.pop(0)
        return self.take_step(self.curpos, self.nextpos)

    def take_step(self, cur, next):
        assert isinstance(cur, tuple)
        assert isinstance(next, tuple)
        #assert (next not in self.board) or (self.board[next]!=0) # next is not a known wall, commented out because can trigger without major harmful effect
        if next[0]==cur[0]-1: 
            if next[1]!=cur[1]:
                raise ValueError(f'take_step invalid from {cur} to {next}')
            return 3 # west
        elif next[0]==cur[0]:
            if next[1]==cur[1]-1:
                return 1 # north
            elif next[1]==cur[1]+1:
                return 2 # south
            else:
                raise ValueError(f'take_step invalid from {cur} to {next}')
        elif next[0]==cur[0]+1: 
            if next[1]!=cur[1]:
                raise ValueError(f'take_step invalid from {cur} to {next}')
            return 4 # east
        else:
            raise ValueError(f'take_step invalid from {cur} to {next}')

    def find_route(self, src, dest):
        assert isinstance(src, tuple)
        assert isinstance(dest, tuple)
        if src==dest:
            return []
        todos=set()
        dist={} # shortest distance from src
        # first paint distances until dest is found
        todos.add(src)
        dist[src]=0
        todo=None
        while len(todos)>0:
            todo=todos.pop()
            if todo==dest:
                break
            assert self.board[todo]!=0
            for st in [(todo[0], todo[1]+1), (todo[0], todo[1]-1), (todo[0]-1, todo[1]), (todo[0]+1, todo[1])]:
                if st in self.board and self.board[st]!=0:
                    if st not in dist or dist[st]>dist[todo]+1:
                        dist[st]=dist[todo]+1
                        todos.add(st)
        if todo!=dest:
            return None
        # now backtrack from dest building up route
        res=[]
        pos=dest
        while pos!=src:
            res.append(pos)
            found=None
            for st in [(pos[0], pos[1]+1), (pos[0], pos[1]-1), (pos[0]-1, pos[1]), (pos[0]+1, pos[1])]:
                if st in dist and dist[st]==dist[pos]-1:
                    found=st
                    break
            assert found is not None
            pos=found
        return res[::-1]

    def put_output(self, a):
        if self.board is None:
            self.init_board()
        assert self.nextpos is not None and isinstance(self.nextpos, tuple)
        if self.nextpos in self.board:
            assert self.board[self.nextpos]==a
        else:
            self.board[self.nextpos]=a
        if a==0: # wall
            self.route=[]
        elif a==1: # corridor
            if self.nextpos not in self.bdist:
                assert self.bdist[self.curpos]==self.todo_dist-1
                self.bdist[self.nextpos]=self.todo_dist
            self.curpos=self.nextpos
        elif a==2: # oxygen system
            if self.nextpos not in self.bdist:
                assert self.bdist[self.curpos]==self.todo_dist-1
                self.bdist[self.nextpos]=self.todo_dist
            self.curpos=self.nextpos
            self.oxypos=self.curpos
            if self.do_part==1:
                self.route=[]
        else:
            assert False
        self.nextpos=None

    def fill_oxygen(self, src):
        assert isinstance(src, tuple)
        todos=set()
        dist={} # shortest distance from src
        # paint distances until completely filled
        todos.add(src)
        dist[src]=0
        todo=None
        while len(todos)>0:
            todo=todos.pop()
            assert self.board[todo]!=0
            for st in [(todo[0], todo[1]+1), (todo[0], todo[1]-1), (todo[0]-1, todo[1]), (todo[0]+1, todo[1])]:
                if st in self.board and self.board[st]!=0:
                    if st not in dist or dist[st]>dist[todo]+1:
                        dist[st]=dist[todo]+1
                        todos.add(st)
        return max(dist.values())

    def count_painted(self):
        return len([ tid for tid in self.board.values() if tid == 2])

    def print_painted(self):
        y_vals=[tup[1] for tup in self.board.keys()]
        x_vals=[tup[0] for tup in self.board.keys()]
        for y in range(min(y_vals), max(y_vals)+1):
            row=''
            for x in range(min(x_vals), max(x_vals)+1):
                c=self.board.get( (x, y) , -1)
                row+={0: '#', 1: '.', 2: 'O', -1: ' '}[c]
            print(row)

sample1=open('data_src/2019-day-15-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ] # program
for do_part in [1,2]:
    print(f'{do_part=}')
    data=collections.defaultdict(lambda : 0)
    for i,v in enumerate(data0): # converted to large mem
        data[i]=v
    cmp=Computer()
    cmp.init_board(do_part)
    cmp.run_opcodes(data)
    if do_part==1:
        print(f'part 1: {cmp.todo_dist=}, {cmp.oxypos=}')
    else:
        t=cmp.fill_oxygen(cmp.oxypos)
        print(f'part 2: {t=}, {cmp.oxypos=}')

# part 1: todo_dist=374
# part 2: t=482

In [ ]:
# 2019 day 14
# start_ts=1665297180
# mv ~/Downloads/input* data_src/2019-day-14-input.txt
# big input file looks like: program
# idea: part 1 parse into tuples, then check 1 fuel rule, DFS backtracking, return nr of ORE

sample1='''
10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL
''' # 31

sample2='''
157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT
''' # 13312 

sample3='''
171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX
''' # 2210736

def count_ore(rule, stock, tamt, rmap):
    '''how much ore are we needing with this rule? we have stock already available
    (count per component), want to produce tamt of our rule element, rmap is
    the total set of rules as map'''
    rnum=rule[-1][0]
    relem=rule[-1][1]
    res=0 # needed ore
    rtimes=(tamt+rnum-1-stock.get(relem, 0))//rnum # how many times to run
    if rtimes<=0:
        stock[relem]=stock.get(relem, 0)-tamt
        assert stock[relem]>=0
        return res
    # we run this rule rtimes but 'in batch'
    for elem in rule[:-1]:
        if elem[1]=='ORE': # only item w/out stock
            res+=elem[0]*rtimes
        else:
            trule=rmap[elem[1]]
            res+=count_ore(trule, stock, elem[0]*rtimes, rmap)
    stock[relem]=stock.get(relem, 0)+rnum*rtimes-tamt
    assert stock[relem]>=0
    return res

sample1=open('data_src/2019-day-14-input.txt').read()
lines=[s.replace(' => ', ', ') for s in sample1.splitlines() if len(s)>0 ]
rules=[ s.split(',') for s in lines ]
rules=[ [ (int(elem.split()[0]), elem.split()[1]) for elem in row ] for row in rules ]
rmap={ row[-1][1]: row for row in rules } # maps target elem to rule, assuming only one rule per element
root=rmap['FUEL']
assert root[-1][0]==1
# part 1
ore=count_ore(root, {}, 1, rmap)
print(f'part 1 {ore=}')
## part 2, binary search
start=1
end=1000000000000
while start<end-1:
    mid=(start+end)//2
    ore=count_ore(root, {}, mid, rmap)
    if ore<1000000000000:
        start=mid
    else:
        end=mid
print('part2')
for mid in [start, end]:
    print(mid, '->', count_ore(root, {}, mid, rmap))

# part 1: 202617
# part 2: 7863863

In [ ]:
# 2019 day 13
# start_ts=1665212872
# mv ~/Downloads/input* data_src/2019-day-13-input.txt
# big input file looks like: intcode with serious data included
# idea: part 1 parse/run intcode, draw outputs, then count

#sample1='''
#'''

class Computer:
    def __init__(self):
        self.relbase=0
        self.i_ptr=0
        self.OFFSET_DIVS={1: 100, 2: 1000, 3: 10000}
        self.board=None

    def fetch_param(self, data, opcode, i, offset):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            return data[param]
        elif mode==1: # immediate mode
            return param
        elif mode==2: # relative mode
            return data[param+self.relbase]
        else:
            print(f'invalid mode {mode}')
            assert False

    def store_param(self, data, opcode, i, offset, newval):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            data[param]=newval
        elif mode==2: # relative mode
            data[param+self.relbase]=newval
        else:
            print(f'invalid mode {mode}')
            assert False

    def run_opcodes(self, data):
        i=self.i_ptr
        halted=False
        while True:
            if data[i]%100==1: # add
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a+b)
                i+=4
            elif data[i]%100==2: # mult
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a*b)
                i+=4
            elif data[i]%100==3: # input
                a=self.get_input()
                self.store_param(data, data[i], i, 1, a)
                i+=2
            elif data[i]%100==4: # output
                a=self.fetch_param(data, data[i], i, 1)
                self.put_output(a)
                i+=2
            elif data[i]%100==5: # jump-if-true
                a=self.fetch_param(data, data[i], i, 1)
                if a!=0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==6: # jump-if-false
                a=self.fetch_param(data, data[i], i, 1)
                if a==0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==7: # less-than
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a<b else 0)
                i+=4
            elif data[i]%100==8: # equals
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a==b else 0)
                i+=4
            elif data[i]%100==9: # relbase
                a=self.fetch_param(data, data[i], i, 1)
                self.relbase+=a
                i+=2
            elif data[i]%100==99: # stop
                halted=True
                break
            else:
                print(f'unknown instruction {data[i]} at position {i}')
                assert False
        self.i_ptr=i
        return

    def init_board(self):
        assert self.board is None
        self.board={} # maps (x,y) cell to tile id
        self.out_q=[]
        self.turns=0
        self.score=None

    def get_input(self):
        if self.board is None:
            self.init_board()
        #if self.turns%100==0:
        #    self.print_painted()
        #if self.turns>1000:
        #    exit()
        self.turns+=1
        return self.best_move()

    def put_output(self, a):
        if self.board is None:
            self.init_board()
        self.out_q.append(a)
        if len(self.out_q)>=3:
            x,y,tid=self.out_q
            if x== -1 and y==0:
                self.score=tid
            else:
                self.board[ (x, y) ]=tid
            self.out_q=[]

    def count_painted(self):
        return len([ tid for tid in self.board.values() if tid == 2])

    def print_painted(self):
        y_vals=[tup[1] for tup in self.board.keys()]
        x_vals=[tup[0] for tup in self.board.keys()]
        for y in range(min(y_vals), max(y_vals)+1):
            row=''
            for x in range(min(x_vals), max(x_vals)+1):
                c=self.board.get( (x, y) , 0)
                row+=str(c) if c!=0 else ' '
            print(row)

    def best_move(self):
        '''move towards the ball'''
        ball_x=None
        paddle_x=None
        for tup,tid in self.board.items():
            if tid==4:
                ball_x=tup[0]
            elif tid==3:
                paddle_x=tup[0]
        assert ball_x is not None
        assert paddle_x is not None
        if ball_x>paddle_x:
            return 1
        elif ball_x<paddle_x:
            return -1
        else:
            return 0

sample1=open('data_src/2019-day-13-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ] # program
for do_part in [1,2]:
    print(f'{do_part=}')
    data=collections.defaultdict(lambda : 0)
    for i,v in enumerate(data0): # converted to large mem
        data[i]=v
    cmp=Computer()
    cmp.init_board()
    if do_part==2:
        data[0]=2
    cmp.run_opcodes(data)
    if do_part==1:
        print(cmp.count_painted())
    else:
        cmp.print_painted()
        print(f'won? {cmp.score=}')

# part 1: 200
# part 2: 9803

In [ ]:
# 2019 day 12 part 1
# start_ts=1664775498
# mv ~/Downloads/input* data_src/2019-day-12-input.txt
# big input file looks like: 4 xyz coords -8 .. 17
# idea: part 1 parse w/re, then simulate

sample1='''
<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>
'''

sample2='''
<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>
'''

def sign(n):
    if n<0:
        return -1
    elif n>0:
        return 1
    else:
        return 0

def do_gravity(pos_data, vel_data):
    for i in range(len(pos_data)):
        for j in range(i+1, len(pos_data)):
            for k in range(3):
                dist=pos_data[i][k]-pos_data[j][k]
                vel_data[i][k]-=sign(dist)
                vel_data[j][k]+=sign(dist)

def do_velocity(pos_data, vel_data):
    for i in range(len(pos_data)):
        for k in range(3):
            pos_data[i][k]+=vel_data[i][k]

def energy(pos_data, vel_data):
    res=0
    for i in range(len(pos_data)):
        pot=sum([abs(n) for n in pos_data[i]])
        kin=sum([abs(n) for n in vel_data[i]])
        res+=pot*kin
    return res

def unlist(tl):
    res=[]
    last=tl[0]
    for n in tl[1:]:
        res.append(n-last)
        last=n
    return tl[0], res

def do_universe(pos_data, vel_data, universe, turn):
    for k in range(3):
        posl=[ll[k] for ll in pos_data]
        vell=[ll[k] for ll in vel_data]
        posvel=tuple(posl+vell)
        tl=universe[k].setdefault( posvel, [])
        tl.append(turn)

sample1=open('data_src/2019-day-12-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
pos_data=[ [int(gs) for gs in result.group(1, 2, 3)] for s in lines if (result:= re.match(r'.*x=([\d-]+)\s*,\s*y=([\d-]+)\s*,\s*z=([\d-]+)', s)) ]
vel_data=[]
for i in pos_data:
    vel_data.append( [0, 0, 0] )
universe=[{}, {}, {}] # list of maps of (pos-m0 pos-m1.. vel-m0 vel-m1) to [turns], i.e. per coord
do_universe(pos_data, vel_data, universe, 0)
for stepno in range(1000):
    do_gravity(pos_data, vel_data)
    do_velocity(pos_data, vel_data)
    do_universe(pos_data, vel_data, universe, stepno+1)
print('energy:', energy(pos_data, vel_data))

# part 1: 9876

In [ ]:
# part 2
# idea: part 2 record list of states separately per x, y and z coords (so e.g. x0 x1 x2 x3 vx0 vx1 vx2 vx3)
# with the turns in which they come up, then you can see their periodicity, apparently every cycle has the same
# length looking at one of the x/y/z components, so the earliest position is the starting position (proof left
# for the reader), then just calculate the least common multiple

# continue running until universe is filled enough so that every coord has repeated at least once
while stepno<1000000:
    stepno+=1
    do_gravity(pos_data, vel_data)
    do_velocity(pos_data, vel_data)
    do_universe(pos_data, vel_data, universe, stepno+1)
periods={} # maps k to first period
for k in range(3):
    print(f'coord {k}')
    kpers=set()
    for st, perl in [unlist(tl) for tl in universe[k].values()]:
        kpers.update(set(perl))
    print(f'{kpers=}')
    assert len(kpers)==1 # apparently always(?) only one period per coordinate
    periods[k]=min(kpers)
vals=list(periods.values())
assert len(vals)==3
lcm1=vals[0]*vals[1]//math.gcd(vals[0], vals[1])
lcm2=lcm1*vals[2]//math.gcd(lcm1, vals[2])
print(f'{lcm2=}')

# part 2: 307043147758488

In [ ]:
# 2019 day 11
# start_ts=1664690374
# mv ~/Downloads/input* data_src/2019-day-11-input.txt
# big input file looks like: intcode
# idea: part 1 parse & run intcode, then manage input/output as a board of coords starting at 0,0 with color None

#sample1='''
#'''

class Computer:
    def __init__(self):
        self.relbase=0
        self.i_ptr=0
        self.OFFSET_DIVS={1: 100, 2: 1000, 3: 10000}
        self.board=None

    def fetch_param(self, data, opcode, i, offset):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            return data[param]
        elif mode==1: # immediate mode
            return param
        elif mode==2: # relative mode
            return data[param+self.relbase]
        else:
            print(f'invalid mode {mode}')
            assert False

    def store_param(self, data, opcode, i, offset, newval):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            data[param]=newval
        elif mode==2: # relative mode
            data[param+self.relbase]=newval
        else:
            print(f'invalid mode {mode}')
            assert False

    def run_opcodes(self, data):
        i=self.i_ptr
        halted=False
        while True:
            if data[i]%100==1: # add
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a+b)
                i+=4
            elif data[i]%100==2: # mult
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a*b)
                i+=4
            elif data[i]%100==3: # input
                a=self.get_input()
                self.store_param(data, data[i], i, 1, a)
                i+=2
            elif data[i]%100==4: # output
                a=self.fetch_param(data, data[i], i, 1)
                self.put_output(a)
                i+=2
            elif data[i]%100==5: # jump-if-true
                a=self.fetch_param(data, data[i], i, 1)
                if a!=0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==6: # jump-if-false
                a=self.fetch_param(data, data[i], i, 1)
                if a==0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==7: # less-than
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a<b else 0)
                i+=4
            elif data[i]%100==8: # equals
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a==b else 0)
                i+=4
            elif data[i]%100==9: # relbase
                a=self.fetch_param(data, data[i], i, 1)
                self.relbase+=a
                i+=2
            elif data[i]%100==99: # stop
                halted=True
                break
            else:
                print(f'unknown instruction {data[i]} at position {i}')
                assert False
        self.i_ptr=i
        return

    def init_board(self):
        assert self.board is None
        self.board={} # maps (x,y) cell to color 0 or 1
        self.rob_x=0
        self.rob_y=0
        self.rob_dir=0 # 0 is up, 1 is right, 2 is down, 3 is left
        self.rob_painting=True # if True output will paint, if False output will turn and move

    def get_input(self):
        if self.board is None:
            self.init_board()
        return self.board.get( (self.rob_x, self.rob_y) , 0)

    def put_output(self, a):
        if self.board is None:
            self.init_board()
        assert 0 <= a <= 1
        if self.rob_painting:
            self.board[ (self.rob_x, self.rob_y) ]=a
            self.rob_painting=False
        else: # turn & moving
            if a==0:
                self.rob_dir-=1
            else:
                self.rob_dir+=1
            self.rob_dir=self.rob_dir % 4
            if self.rob_dir==0:
                self.rob_y-=1
            elif self.rob_dir==1:
                self.rob_x+=1
            elif self.rob_dir==2:
                self.rob_y+=1
            elif self.rob_dir==3:
                self.rob_x-=1
            else:
                assert False
            self.rob_painting=True

    def count_painted(self):
        return len(self.board)

    def print_painted(self):
        y_vals=[tup[1] for tup in self.board.keys()]
        x_vals=[tup[0] for tup in self.board.keys()]
        for y in range(min(y_vals), max(y_vals)+1):
            row=''
            for x in range(min(x_vals), max(x_vals)+1):
                c=self.board.get( (x, y) , 0)
                row+='X' if c==1 else ' '
            print(row)

sample1=open('data_src/2019-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ] # program
for do_part in [1, 2]:
    print(f'{do_part=}')
    data=collections.defaultdict(lambda : 0)
    for i,v in enumerate(data0): # converted to large mem
        data[i]=v
    cmp=Computer()
    cmp.init_board()
    if do_part==2:
        cmp.board[ (0,0) ]=1
    cmp.run_opcodes(data)
    if do_part==1:
        print(cmp.count_painted())
    elif do_part==2:
        cmp.print_painted()

# part 1: 2293
# part 2: AHLCPRAL

In [ ]:
# TEMPLATE
# 2019 day 6
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2019-day-6-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample1='''

'''

#sample1=open('data_src/2019-day-6-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ int(s) for s in lines[0].split(',') ]
groups=get_line_groups(lines)
data0=[ s.split() for s in lines ]
data0=[ [cmd, int(num), 0] for cmd, num in data0 ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed